In [2]:
import pandas as pd
import numpy as np

directory of meta data ftp://ftp.ncbi.nlm.nih.gov/sra/reports/Metadata

### input directory

In [3]:

basePickleDir='/data/cellardata/users/btsui/SRA/DUMP/'
accessionProjDir='/data/cellardata/users/btsui/SRA/DUMP/SRA_Accessions.tab'
accessionDir='/data/cellardata/users/btsui/SRA/DUMP/SRA_Run_Members.tab'

In [4]:
srsMergedS=pd.read_pickle(basePickleDir+'allSRS.pickle.gz')
srxMergedS=pd.read_pickle(basePickleDir+'allSRX.pickle.gz')

### output directory

In [5]:
sra_dump_csv_dir='/cellar/users/btsui/Data/SRA/DUMP/sra_dump.csv.gz'
sra_dump_pickle_dir='/cellar/users/btsui/Data/SRA/DUMP/sra_dump.pickle'


In [6]:
projAccessDf=pd.DataFrame.from_csv(accessionProjDir,sep='\t')

/cellar/users/btsui/anaconda3/envs/deep_nlp_cpu/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: from_csv is deprecated. Please use read_csv(...) instead. Note that some of the default arguments are different, so please refer to the documentation for from_csv when changing your function calls
  """Entry point for launching an IPython kernel.
/cellar/users/btsui/anaconda3/envs/deep_nlp_cpu/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2901: DtypeWarning: Columns (13,14,15) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


In [7]:
accessionDf=pd.DataFrame.from_csv(accessionDir,sep='\t')

/cellar/users/btsui/anaconda3/envs/deep_nlp_cpu/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: from_csv is deprecated. Please use read_csv(...) instead. Note that some of the default arguments are different, so please refer to the documentation for from_csv when changing your function calls
  """Entry point for launching an IPython kernel.
/cellar/users/btsui/anaconda3/envs/deep_nlp_cpu/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2901: DtypeWarning: Columns (5,6) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


### filter onto only human mouse, and zebra fish data

In [8]:
validScientificNameSrs=srsMergedS[srsMergedS.index.get_level_values(1)=='SCIENTIFIC_NAME']#[scientific_nameS.isin({'Homo sapiens','Mus musculus'})]
#rename the genomes to match the data

In [9]:
#validSrs=validScientificNameSrs.index.get_level_values(0)

In [10]:
srsToSpecies=pd.Series( index=validScientificNameSrs.index.get_level_values(0),data=validScientificNameSrs.values)

uniqueSrsToSpeciesS=srsToSpecies.groupby(srsToSpecies.index).first()

### filter onto to RNA-seq

In [11]:
srxMergedS.head(n=6)

ERX1269638  TITLE                Illumina HiSeq 1500 paired end sequencing
            LIBRARY_STRATEGY                                           EST
            LIBRARY_SOURCE                                         GENOMIC
            LIBRARY_SELECTION                                       RANDOM
            LIBRARY_LAYOUT                                          PAIRED
ERX1269639  TITLE                Illumina HiSeq 1500 paired end sequencing
dtype: object

In [12]:
layoutS=srxMergedS[srxMergedS.index.get_level_values(1)=='LIBRARY_LAYOUT'].groupby(level=0).first()

libraryS=srxMergedS[srxMergedS.index.get_level_values(1)=='LIBRARY_STRATEGY'].groupby(level=0).first()

In [13]:
srxToSeqS =libraryS.groupby(level=0).first()

In [14]:
validSrx=srxToSeqS.index.get_level_values(0)

## take only runs that satisfy both srx and srs filter

In [15]:
subAccessionDf=accessionDf[accessionDf.Sample.isin(uniqueSrsToSpeciesS.index)&accessionDf.Experiment.isin(srxToSeqS.index)]

In [16]:
subAccessionDf.loc[:,'ScientificName']=uniqueSrsToSpeciesS.loc[subAccessionDf.Sample.values].values[:]

/cellar/users/btsui/anaconda3/envs/deep_nlp_cpu/lib/python3.6/site-packages/pandas/core/indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
/cellar/users/btsui/anaconda3/envs/deep_nlp_cpu/lib/python3.6/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [17]:
subAccessionDf['Run']=subAccessionDf.index

/cellar/users/btsui/anaconda3/envs/deep_nlp_cpu/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [18]:
publicRunDf=subAccessionDf

In [19]:
algnedSrx=srxToSeqS.loc[publicRunDf.Experiment].values
srrToSeqS=pd.Series(data=algnedSrx,index=publicRunDf.index)
publicRunDf.loc[:,'LibraryStrategy']=srrToSeqS[:]

/cellar/users/btsui/anaconda3/envs/deep_nlp_cpu/lib/python3.6/site-packages/pandas/core/indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
/cellar/users/btsui/anaconda3/envs/deep_nlp_cpu/lib/python3.6/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [20]:
publicRunDf['LibraryLayout']=layoutS.loc[publicRunDf['Experiment'].values].values

/cellar/users/btsui/anaconda3/envs/deep_nlp_cpu/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [21]:
projectSubDf=projAccessDf.loc[publicRunDf.index]
projectSubDf.columns='proj_accession_'+projectSubDf.columns

/cellar/users/btsui/anaconda3/envs/deep_nlp_cpu/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  """Entry point for launching an IPython kernel.


In [22]:
yearAnnotatedPublicDf=pd.concat( [publicRunDf,projectSubDf],axis=1)#.iloc[8]

In [23]:
yearAnnotatedPublicDf['ScientificName']=yearAnnotatedPublicDf['ScientificName'].str.replace(' ','_')

In [37]:
import string
printable = set(string.printable)

removeNonAscii=lambda s: "".join(filter(lambda x: x in printable, s))
#removeNonAsciiCol=lambda col:col.apply(removeNonAscii)

In [55]:
subExportDf=yearAnnotatedPublicDf.loc[:,yearAnnotatedPublicDf.columns!='Run']


In [56]:
#subExportDf['ScientificName']

In [57]:
#### convert the micrbiome to something else

In [58]:
subExportDf['new_ScientificName']=subExportDf['ScientificName']

In [59]:
m_4=subExportDf['new_ScientificName'].str.contains('human')
#Homo_sapiens
subExportDf.loc[m_4, 'new_ScientificName']='Homo_sapiens'
#m_4=subExportDf['ScientificName'].str.contains('metagenome')
m_5=subExportDf['new_ScientificName'].str.contains('mouse')
#Homo_sapiens
subExportDf.loc[m_5, 'new_ScientificName']='Mus_musculus'
###for soil, to eliminate human contamination, align to human reference
m_6=subExportDf['new_ScientificName'].str.contains('^soil_metagenome#')
subExportDf.loc[m_6, 'new_ScientificName']='Homo_sapiens'




In [60]:
m_7=subExportDf['new_ScientificName'].str.contains('^Canis')
subExportDf.loc[m_7, 'new_ScientificName']='Canis_familiaris'

In [61]:
for myCol in [u'ScientificName', u'LibraryStrategy',
       u'LibraryLayout','new_ScientificName']:
    subExportDf[myCol]=subExportDf[myCol].apply(removeNonAscii)

In [62]:
subExportDf.head()

Member_Name  Experiment      Sample Study    Spots      Bases  \
Run                                                                        
ERR1823524     default  ERX1886028  ERS1545929     -  1167564  426746666   
ERR1823525     default  ERX1886029  ERS1545930     -   498259  207579474   
ERR1823526     default  ERX1886030  ERS1545931     -   584246  225394117   
ERR1823527     default  ERX1886031  ERS1545932     -   708096  316102995   
ERR1823528     default  ERX1886032  ERS1545933     -  1031600  442097283   

           Status      BioSample         ScientificName LibraryStrategy  \
Run                                                                       
ERR1823524   live  SAMEA80860918  Staphylococcus_aureus             WGS   
ERR1823525   live  SAMEA80861668  Staphylococcus_aureus             WGS   
ERR1823526   live  SAMEA80862418  Staphylococcus_aureus             WGS   
ERR1823527   live  SAMEA80863168  Staphylococcus_aureus             WGS   
ERR1823528   live  SAMEA80863918  Staphylococcus_aureus             WGS   

                    ...           proj_accession_Sample proj_accession_Study  \
Run                 ...                                                        
ERR1823524          ...                      ERS1545929                    -   
ERR1823525          ...                      ERS1545930                    -   
ERR1823526          ...                      ERS1545931                    -   
ERR1823527          ...                      ERS1545932                    -   
ERR1823528          ...                      ERS1545933                    -   

           proj_accession_Loaded proj_accession_Spots proj_accession_Bases  \
Run                                                                          
ERR1823524                     1              1167564            426746666   
ERR1823525                     1               498259            207579474   
ERR1823526                     1               584246            225394117   
ERR1823527                     1               708096            316102995   
ERR1823528                     1              1031600            442097283   

                       proj_accession_Md5sum proj_accession_BioSample  \
Run                                                                     
ERR1823524  30e8c78e7dc61ce33100b0f62b80024a            SAMEA80860918   
ERR1823525  a6ca66769b455d8803633120065ad2db            SAMEA80861668   
ERR1823526  e7e469e8d3f5ed098bdc569ca648e61e            SAMEA80862418   
ERR1823527  01bad3c0fa4bdc24ab1017443ce1fce1            SAMEA80863168   
ERR1823528  428ab0594732b2a707ff46700a890dd5            SAMEA80863918   

           proj_accession_BioProject proj_accession_ReplacedBy  \
Run                                                              
ERR1823524                         -                         -   
ERR1823525                         -                         -   
ERR1823526                         -                         -   
ERR1823527                         -                         -   
ERR1823528                         -                         -   

               new_ScientificName  
Run                                
ERR1823524  Staphylococcus_aureus  
ERR1823525  Staphylococcus_aureus  
ERR1823526  Staphylococcus_aureus  
ERR1823527  Staphylococcus_aureus  
ERR1823528  Staphylococcus_aureus  

[5 rows x 31 columns]

In [ ]:
subExportDf.to_csv(sra_dump_csv_dir,compression='gzip')

### create smaller version for pickle

In [ ]:
pickleExportInDf=subExportDf.copy()

In [ ]:
timeCols=['proj_accession_Updated'
          ,'proj_accession_Published'
          ,'proj_accession_Received']
for timeCol in timeCols:
    tmp_dateS=pickleExportInDf[timeCol].str.split('T').str[0]
    pickleExportInDf[timeCol]=pd.to_datetime( tmp_dateS)

In [ ]:
ignoreCols=['proj_accession_BioProject','proj_accession_Study','proj_accession_Spots','proj_accession_BioSample'
           ,'proj_accession_Sample','proj_accession_Experiment','proj_accession_Alias','proj_accession_Md5sum',
            'BioSample','proj_accession_Bases']

In [ ]:
subsetPickleDf=pickleExportInDf.loc[:,~pickleExportInDf.columns.isin(ignoreCols)]

In [ ]:
intTypes=['Spots','Bases']
for intType in intTypes:
    subsetPickleDf[intType]=pd.to_numeric(subsetPickleDf[intType],errors='coerce')

In [ ]:
factorTypes=[u'proj_accession_ReplacedBy', u'proj_accession_Type',
       u'proj_accession_Status', u'Status', u'proj_accession_Visibility',
       u'LibraryLayout', u'proj_accession_Loaded', u'LibraryStrategy',
       u'proj_accession_Updated', u'proj_accession_Received',
       u'proj_accession_Published', u'Member_Name', u'proj_accession_Center',
       u'ScientificName','new_ScientificName' ]#u'Study', u'proj_accession_Submission'

In [ ]:
for factorType in factorTypes:
    subsetPickleDf[factorType]=subsetPickleDf[factorType].astype('category')

In [ ]:
#sra_dump_csv_dir,sra_dump_pickle_dir
#subsetPickleDf.to_pickle(sra_dump_pickle_dir)

### spit in the extracted ids

In [53]:
import pandas as pd
import numpy as np
subsetPickleDf=pd.read_pickle(sra_dump_pickle_dir)

In [54]:
#subsetPickleDf.dtypes

In [55]:
#['Run_db','Run_digits']
subsetPickleDf['Run_db']=subsetPickleDf.index.str.extract('([EDS]RR)').values
subsetPickleDf['Run_digits']=subsetPickleDf.index.str.extract('[EDS]RR(\d+)').astype(np.int)

/cellar/users/btsui/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:2: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
  
/cellar/users/btsui/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:3: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
  This is separate from the ipykernel package so we can avoid doing imports until


In [56]:
#subsetPickleDf.dtypes

In [57]:
subsetPickleDf.to_pickle(sra_dump_pickle_dir)

### scratch

In [ ]:
new_size=249733376L
old_size=1035668000L
new_size/float(old_size)

In [ ]:
scentific_VC=subExportDf['ScientificName'].value_counts()

In [43]:
#subExportDf

In [70]:
#scentific_VC.to_frame()

In [71]:
#scentific_VC[scentific_VC.index.str.contains('human')].iloc[:20]

In [72]:
#scentific_VC[scentific_VC.index.str.contains('mouse')].iloc[:20]